# 第7章 機械学習の前処理を行う10本ノック

In [41]:
ENV["LINES"] = 10

10

In [147]:
using DataFrames
using DataFramesMeta
using CSV
using MLJ

## 81

In [15]:
dataset_df = CSV.read(download("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv"), DataFrame)

Row,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
,Int64,Int64,String7,Float64?,Int64,Int64,Float64,String1?,String7,String7,Bool,String1?,String15?,String3,Bool
1,0,3,male,22.0,1,0,7.25,S,Third,man,true,missing,Southampton,no,false
2,1,1,female,38.0,1,0,71.2833,C,First,woman,false,C,Cherbourg,yes,false
3,1,3,female,26.0,0,0,7.925,S,Third,woman,false,missing,Southampton,yes,true
4,1,1,female,35.0,1,0,53.1,S,First,woman,false,C,Southampton,yes,false
5,0,3,male,35.0,0,0,8.05,S,Third,man,true,missing,Southampton,no,true
6,0,3,male,missing,0,0,8.4583,Q,Third,man,true,missing,Queenstown,no,true
7,0,1,male,54.0,0,0,51.8625,S,First,man,true,E,Southampton,no,true
8,0,3,male,2.0,3,1,21.075,S,Third,child,false,missing,Southampton,no,false
9,1,3,female,27.0,0,2,11.1333,S,Third,woman,false,missing,Southampton,yes,false


In [87]:
label = dataset_df.survived
dataset2_df = select(dataset_df, Not(:survived))

Row,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
,Int64,String7,Float64?,Int64,Int64,Float64,String1?,String7,String7,Bool,String1?,String15?,String3,Bool
1,3,male,22.0,1,0,7.25,S,Third,man,true,missing,Southampton,no,false
2,1,female,38.0,1,0,71.2833,C,First,woman,false,C,Cherbourg,yes,false
3,3,female,26.0,0,0,7.925,S,Third,woman,false,missing,Southampton,yes,true
4,1,female,35.0,1,0,53.1,S,First,woman,false,C,Southampton,yes,false
5,3,male,35.0,0,0,8.05,S,Third,man,true,missing,Southampton,no,true
6,3,male,missing,0,0,8.4583,Q,Third,man,true,missing,Queenstown,no,true
7,1,male,54.0,0,0,51.8625,S,First,man,true,E,Southampton,no,true
8,3,male,2.0,3,1,21.075,S,Third,child,false,missing,Southampton,no,false
9,3,female,27.0,0,2,11.1333,S,Third,woman,false,missing,Southampton,yes,false


## 82

In [88]:
(train_df, test_df), (train_label, test_label) = partition((dataset2_df, label), 0.7; rng=1234, stratify=label, multi=true)

((623×14 DataFrame
 Row │ pclass  sex      age        sibsp  parch  fare     embarked  class    w ⋯
     │ Int64   String7  Float64?   Int64  Int64  Float64  String1?  String7  S ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │      2  male          51.0      0      0  12.525   S         Second   m ⋯
  ⋮  │   ⋮        ⋮         ⋮        ⋮      ⋮       ⋮        ⋮         ⋮       ⋱
                                                  6 columns and 622 rows omitted, 268×14 DataFrame
 Row │ pclass  sex      age        sibsp  parch  fare     embarked  class    w ⋯
     │ Int64   String7  Float64?   Int64  Int64  Float64  String1?  String7  S ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │      1  female        24.0      0      0  83.1583  C         First    w ⋯
  ⋮  │   ⋮        ⋮         ⋮        ⋮      ⋮       ⋮        ⋮         ⋮       ⋱
                                                  6 columns and 267 rows

## 83

In [100]:
select!(train_df, Not([:embark_town, :alive]))

ArgumentError: ArgumentError: column name :embark_town not found in the data frame

In [101]:
ux = unique(train_df.sex); DataFrames.transform(train_df, @. :sex => ByRow(isequal(ux)) .=> Symbol(:sex_, ux))

ArgumentError: ArgumentError: column name :sex not found in the data frame

In [102]:
unique_class = unique(train_df.class)
unique_class_d = Dict(zip(unique_class, 1:length(unique_class)))
DataFrames.transform(train_df, :class => ByRow(x -> unique_class_d[x]) => :class)

ArgumentError: ArgumentError: column name "class" not found in the data frame; existing most similar names are: "pclass"

In [131]:
function delete_column(df, column_l)
    available_column_l = []
    for column_i in column_l
        if column_i in names(df)
            append!(column_i)
        end
    end
    if length(available_column_l)
        return df
    end
    return select(df, Not(available_column_l))
end
function get_dummies(df)
    column_l = names(train_df, Union{AbstractString, Missing})
    column_l = vcat(column_l, "pclass")
    for column_i in column_l
        unique_l = unique(df[!, Symbol(column_i)])
        df = DataFrames.transform(df, @. Symbol(column_i) => ByRow(isequal(unique_l)) .=> Symbol("$(column_i)_", unique_l))
        select!(df, Not(column_i))
    end
    return df
end

get_dummies (generic function with 1 method)

In [133]:
train_df = get_dummies(train_df)

Row,age,sibsp,parch,fare,adult_male,alone,sex_male,sex_female,embarked_S,embarked_C,embarked_Q,embarked_missing,class_Second,class_Third,class_First,who_man,who_child,who_woman,deck_missing,deck_C,deck_A,deck_B,deck_D,deck_E,deck_F,deck_G,pclass_2,pclass_3,pclass_1
,Float64?,Int64,Int64,Float64,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool
1,51.0,0,0,12.525,true,true,true,false,true,false,false,false,true,false,false,true,false,false,true,false,false,false,false,false,false,false,true,false,false
2,28.0,0,0,22.525,true,true,true,false,true,false,false,false,false,true,false,true,false,false,true,false,false,false,false,false,false,false,false,true,false
3,13.0,0,0,7.2292,false,true,false,true,false,true,false,false,false,true,false,false,true,false,true,false,false,false,false,false,false,false,false,true,false
4,missing,0,0,8.05,true,true,true,false,true,false,false,false,false,true,false,true,false,false,true,false,false,false,false,false,false,false,false,true,false
5,missing,2,0,21.6792,true,false,true,false,false,true,false,false,false,true,false,true,false,false,true,false,false,false,false,false,false,false,false,true,false
6,59.0,0,0,13.5,true,true,true,false,true,false,false,false,true,false,false,true,false,false,true,false,false,false,false,false,false,false,true,false,false
7,20.0,0,0,7.8542,true,true,true,false,true,false,false,false,false,true,false,true,false,false,true,false,false,false,false,false,false,false,false,true,false
8,49.0,1,1,110.883,true,false,true,false,false,true,false,false,false,false,true,true,false,false,false,true,false,false,false,false,false,false,false,false,true
9,21.0,0,0,8.05,true,true,true,false,true,false,false,false,false,true,false,true,false,false,true,false,false,false,false,false,false,false,false,true,false


In [152]:
iqr_df = @chain train_df begin
    describe(:q25, :q75)
    @transform begin
        @astable begin
            :iqr = :q75 - :q25
            :mx = :q75 + 1.5 * :iqr
            :mn = :q25 - 1.5 * :iqr
        end
    end
    @select(:variable, :mx, :mn)
end

Row,variable,mx,mn
,Symbol,Float64,Float64
1,age,65.0,-7.0
2,sibsp,2.5,-1.5
3,parch,0.0,0.0
4,fare,66.3,-27.1
5,adult_male,2.5,-1.5
6,alone,2.5,-1.5
7,sex_male,2.5,-1.5
8,sex_female,2.5,-1.5
9,embarked_S,1.75,-0.25


In [189]:
for column_i in names(train_df)
    idx = iqr_df.variable .== :age
    # minimum
    smn = sum(skipmissing(train_df[!, Symbol(column_i)] .<= iqr_df[idx, :mn]))
    # max
    smx = sum(skipmissing(train_df[!, Symbol(column_i)] .>= iqr_df[idx, :mx]))
    println(column_i, " ", smn + smx)
end

age 9


sibsp 0
parch 0
fare 84
adult_male 0
alone 0
sex_male 0
sex_female 0
embarked_S 0
embarked_C 0
embarked_Q 0
embarked_missing 0
class_Second 0
class_Third 0
class_First 0
who_man 0
who_child 0
who_woman 0
deck_missing 0
deck_C 0
deck_A 0
deck_B 0
deck_D 0
deck_E 0
deck_F 0
deck_G 0
pclass_2 0
pclass_3 0
pclass_1 0


## 85